In [1]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
import time
import shutil
from shapely.geometry import Point, Polygon,box
import numpy as np

In [2]:
occurrence_dir=r'J:\4_GSW\occurrence'
osw_file_list=os.listdir(occurrence_dir)
print(osw_file_list)

['occurrence_0E10E_0N10N.tif', 'occurrence_0E10E_10N20N.tif', 'occurrence_0E10E_10S0N.tif', 'occurrence_0E10E_20N30N.tif', 'occurrence_0E10E_30N40N.tif', 'occurrence_0E10E_40N50N.tif', 'occurrence_0E10E_50N60N.tif', 'occurrence_0E10E_60N70N.tif', 'occurrence_100E110E_0N10N.tif', 'occurrence_100E110E_10N20N.tif', 'occurrence_100E110E_10S0N.tif', 'occurrence_100E110E_20N30N.tif', 'occurrence_100E110E_30N40N.tif', 'occurrence_100E110E_40N50N.tif', 'occurrence_100E110E_50N60N.tif', 'occurrence_100E110E_60N70N.tif', 'occurrence_100E110E_70N80N.tif', 'occurrence_100W90W_0N10N.tif', 'occurrence_100W90W_10N20N.tif', 'occurrence_100W90W_10S0N.tif', 'occurrence_100W90W_20N30N.tif', 'occurrence_100W90W_30N40N.tif', 'occurrence_100W90W_40N50N.tif', 'occurrence_100W90W_50N60N.tif', 'occurrence_100W90W_60N70N.tif', 'occurrence_100W90W_70N80N.tif', 'occurrence_10E20E_0N10N.tif', 'occurrence_10E20E_0N10N.tif.aux.xml', 'occurrence_10E20E_0N10N.tif.ovr', 'occurrence_10E20E_10N20N.tif', 'occurrence_10E20

In [10]:
mask_dir=r'J:\4_GSW\occurrence_afm'
os.makedirs(mask_dir, exist_ok=True)
i=1
for file in osw_file_list:
    if file.endswith('.tif'):
        print(f'{i}:{file}')
        osw_file=os.path.join(occurrence_dir,file)
        raw_prediction_image=rasterio.open(osw_file)
        out_osw_dir=os.path.join(mask_dir,file)
        bbox = box(*raw_prediction_image.bounds)
        meta=raw_prediction_image.meta
        meta.update({"compress": 'lzw'})
        meta.update({"nodata": 255})
        SHIFT_mask=gps.read_file(r'D:\postprocess\v240220\auxiliary_dataset.gdb',layer='SHIFT',bbox=bbox) 
        mask_patch_geojson = SHIFT_mask.geometry.values 
        if len(mask_patch_geojson) > 0:
            occurrence_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=False)
            print(f'Mask finished:{file}')
        with rasterio.open(out_osw_dir, "w", **meta) as prediction_masked:
            prediction_masked.write(occurrence_masked)
        i=i+1


1:occurrence_0E10E_0N10N.tif
Mask finished:occurrence_0E10E_0N10N.tif
2:occurrence_0E10E_10N20N.tif


KeyboardInterrupt: 

In [7]:
print(meta)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 40000, 'height': 40000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, 0.0,
       0.0, -0.00025, 10.0), 'compress': 'lzw'}
